In [1]:
from searchtweets import load_credentials, collect_results
import pandas as pd
import json
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import datetime
from pandas._libs.tslibs.timestamps import Timestamp
import pickle

import preprocessing_functions as pfunctions
#import warnings
#warnings.filterwarnings('ignore')

# Extraction for all the years

In [3]:
query = {}
query['query'] = '(((venezolano OR venezolana OR venezolanos OR venezolanas OR venezuela OR vzla OR vnzla) (migrante OR migrantes OR migracion)) OR veneco OR veneca OR venecos OR venecas)'
query['max_results'] = 500
search_args = load_credentials(filename="./.twitter_keys.yaml",
                 yaml_key="search_tweets_api",
                 env_overwrite=False)

In [5]:
n_tweets = 250000
start_year = 2014
end_year =2021

In [ ]:
start = dt.date(2014,1,1)
end = dt.date(2014,1,31)
data_ranges = pd.date_range(start= start, end = end, freq="M")

for date_end in data_ranges:
    date_start = datetime.date(date_end.year,date_end.month,1)
    date_start = Timestamp(date_start)
    print(str(date_start.asm8)+" "+str(date_end.asm8))

In [9]:
for i in range(start_year, end_year+1):
    for j in range(1,13):
        i_str = str(i)
        j_str = str(j).zfill(2)
        j1_str = str(j+1).zfill(2)
            
        #Query time
        start_time = i_str+'-'+j_str+'-01T00:00:00.0-05:00'
        if (j==9):
            end_time = i_str+'-'+j_str+'-22T00:00:00.0-05:00'
        else:
            end_time = i_str+'-'+j1_str+'-01T00:00:00.0-05:00'
        query['start_time'] = start_time
        query['end_time'] = end_time
        
        #obtaining tweets
        tweets = collect_results(query,
                         max_tweets=n_tweets, #Número real de tweets a extraer
                         result_stream_args=search_args)
        #json exporting
        json_path = './Tweets/'+i_str+'-'+j_str+'.json'
        out_file = open(json_path, "w") 
        json.dump(tweets, out_file) 
        out_file.close()
        
        #measuring the number of tweets per month
        n_ij = 0
        for k in range(len(tweets)):
            n_ij = n_ij+len(tweets[k]['data'])
        #n_list.insert(1000,n_ij)
        #n_total=sum(n_list)
        
        print("Exported "+str(n_ij)+' tweets at: '+json_path)

Exported 77342 tweets at: ./Tweets/2021-09.json


#### *Note: extracted up to Sept/22/2021*

In [22]:
n_list = []
for i in range(2014, 2022):
    for j in range(1,13):
        i_str = str(i)
        j_str = str(j).zfill(2)
        m = json.load(open("./Tweets/"+i_str+"-"+j_str+".json"))
        n_ij = 0
        for k in range(len(m)):
            n_ij = n_ij+len(m[k]['data'])
        n_list.insert(1000,n_ij)
        n_total=sum(n_list)
sum(n_list)

FileNotFoundError: [Errno 2] No such file or directory: './Tweets/2021-10.json'

# JSON importing all tweets and converting to dataframe

## User tags with tweepy

In [2]:
import tweepy
consumer_key = "OeGkvUry0wD6vA1S5rSxMQVEC" #1
consumer_secret = "6YAlccLnr5C7zASOMEoFRBqqlxwzGeaHM5PTlXGdZ3an80dfJL" #2
access_token = "126698065-NlOxKEjkymKwZae0kGN5dx2rpxkGZX4CFf5bG3pG" #3
access_token_secret = "KLc2hdBXvJJEPXpJZ9Is1oX7u3R55bARO8Q9y9jyTFybf" #4

def tweet_json_find (search_id):
    test_status = api.get_status(search_id, tweet_mode='extended')._json 
    return(test_status)
    
# User data
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# With this object we will call the Twitter API
api = tweepy.API(auth,
                 wait_on_rate_limit=True)
                 

## Geocoding function

In [4]:
geocoder = RateLimiter(Nominatim(user_agent='tweets_project').geocode, min_delay_seconds=1/60,
 return_value_on_exception="no_geolocation", max_retries=0,swallow_exceptions=False)

In [1]:
data_ranges = date_range_gen(2020,5, 2020,5)

for date_i in data_ranges:
    tweets_dict = []
    startTime = time.time()
    
    i_str = str(date_i.year)
    j_str = str(date_i.month).zfill(2)
    m = json.load(open("./Tweets/"+i_str+"-"+j_str+".json")) #open every json file
        
    for k in range(len(m)):
        for l in range(len(m[k]['data'])):
            id_i = m[k]['data'][l]['id']    
            location_i = "no_location"
  
            try:
                json_i = tweet_loc_find(id_i)
                text_i = json_i['full_text']
                location_i = json_i['user']['location']   
            except Exception as e:
                json_i = e
                text_i = m[k]['data'][l]['text']
            
            if location_i != "no_location":
                try: 
                    location_i = geocoder(location_i)
                except Exception as e:
                    pass
             
            tweets_dict.append({'Id':id_i, 'text':text_i, 'json':json_i, 'location':location_i})

        print("Extracted info from", str(len(tweets_dict)), 'tweets')
        pfunctions.print_progress(k+1,len(m),len(m[k]['data'])) # Progress info (optional)
    
    executionTime = (time.time() - startTime)

    times_df.loc[len(times_df['date'])]=[(i_str+"-"+j_str),executionTime]
    tweets_df = pd.DataFrame.from_dict(tweets_dict)
    pickle_out = open("./Tweets/df"+i_str+"-"+j_str+".pickle","wb")
    pickle.dump(tweets_df,pickle_out)
    pickle_out.close
    print(str(date_i)," correctly converted to dataframe and exported in ",round(executionTime,4)," sg", sep = "")
    
times_df.to_csv('./Tweets/times.csv', index = False)


KeyboardInterrupt



In [5]:
times_df = pd.read_csv('./Tweets/times.csv')

In [6]:
data_range = ['2020-05'] #must be in the YYYY-MM format

for date_i in data_range:
    tweets_dict = []
    startTime = time.time()

    m = json.load(open("./Tweets/"+date_i+".json")) #open every json file

    tweets_dict = pfunctions.get_data_and_geo(m, geocoder, tweet_json_find)    #get json data and gelocation info from tweet json
    
    executionTime = (time.time() - startTime)

    times_df.loc[len(times_df['date'])]=[date_i,executionTime]
    tweets_df = pd.DataFrame.from_dict(tweets_dict)
    pickle_out = open("./Tweets/df"+date_i+".pickle","wb")
    pickle.dump(tweets_df,pickle_out)
    pickle_out.close
    print(str(date_i)," correctly converted to dataframe and exported in ",round(executionTime,4)," sg", sep = "")
    
times_df.to_csv('./Tweets/times.csv', index = False)

Extracted info from 462 tweets
For this page, converted 462 tweets: 0.27% of the month in total (pred remaining: 54.54 h)


Rate limit reached. Sleeping for: 328


Extracted info from 931 tweets
For this page, converted 469 tweets: 0.53% of the month in total (pred remaining: 54.4 h)
Extracted info from 1412 tweets
For this page, converted 481 tweets: 0.8% of the month in total (pred remaining: 54.25 h)


Rate limit reached. Sleeping for: 402


Extracted info from 1902 tweets
For this page, converted 490 tweets: 1.07% of the month in total (pred remaining: 54.1 h)
Extracted info from 2386 tweets
For this page, converted 484 tweets: 1.33% of the month in total (pred remaining: 53.96 h)


...

Rate limit reached. Sleeping for: 383


Extracted info from 180497 tweets
For this page, converted 448 tweets: 98.93% of the month in total (pred remaining: 0.58 h)
Extracted info from 180916 tweets
For this page, converted 419 tweets: 99.2% of the month in total (pred remaining: 0.44 h)
Extracted info from 181321 tweets
For this page, converted 405 tweets: 99.47% of the month in total (pred remaining: 0.29 h)


Rate limit reached. Sleeping for: 295


Extracted info from 181709 tweets
For this page, converted 388 tweets: 99.73% of the month in total (pred remaining: 0.15 h)
Extracted info from 182132 tweets
For this page, converted 423 tweets: 100.0% of the month in total (pred remaining: 0.0 h)


ValueError: cannot set a row with mismatched columns

In [7]:
pickle_out = open("./Tweets/df"+date_i+".pickle","wb")
pickle.dump(tweets_df,pickle_out)
pickle_out.close

NameError: name 'tweets_df' is not defined

# JSON convert to dataframe

In [8]:
for i in range(len(tweets_r)):
    for j in range(len(tweets_r[i]['data'])):
        id_i = tweets_r[i]['data'][j]['id']
        text_i = tweets_r[i]['data'][j]['text']
        tweets_df.loc[len(tweets_df['Id'])]=[id_i,text_i]      

In [9]:
tweets_df = tweets_df.drop(0)


In [10]:
tweets_df = tweets_df.reset_index()

In [19]:
tweets_df

,index,Id,text,json
0,1,429478735541460992,RT @Veneco_Babes: @marycarmenfh @trinysex @Div...,{'created_at': 'Sat Feb 01 04:58:04 +0000 2014...
1,2,429476144170422272,RT @Veneco_Babes: @marycarmenfh @trinysex @Div...,amarillo
2,3,429475247448793088,@aluxtuitea @rogeliosamour @JJRENDON @JNatanVa...,amarillo
3,4,429443798234824704,@Mayomendozaf @callollanos @JanioCuadro el que...,amarillo
4,5,429438621037625344,A ustedes les gusta que les digan Venecos/as?,amarillo
...,...,...,...,...
1722,1723,418300487445012480,Veneco &amp; Young King ♥ https://t.co/lMYzGRUxwr,amarillo
1723,1724,418277758989385728,Los tios venecos #familia #familia #navidades2...,amarillo
1724,1725,418262764000665600,@khristopbarca jajajajajaj y a que mundial es ...,amarillo
1725,1726,418256318571753472,"Feliz año para ti mi amor, i love RT @k_mpillo...",amarillo


In [51]:
tweet_loc_find(429478735541460992)['retweeted_status']

{'created_at': 'Thu Jan 30 19:05:32 +0000 2014',
 'id': 428967230689517568,
 'id_str': '428967230689517568',
 'full_text': '@marycarmenfh @trinysex @DivaCandances @giselleavendao @LorenaParisella @carlaangola @patitolocurita1 Buenas tardes #BellezasTwitteras',
 'truncated': False,
 'display_text_range': [0, 134],
 'entities': {'hashtags': [{'text': 'BellezasTwitteras',
    'indices': [116, 134]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'marycarmenfh',
    'name': 'Mary Carmen Fariña',
    'id': 114949779,
    'id_str': '114949779',
    'indices': [0, 13]},
   {'screen_name': 'trinysex',
    'name': 'Profesexual (+18)',
    'id': 3019078349,
    'id_str': '3019078349',
    'indices': [14, 23]},
   {'screen_name': 'giselleavendao',
    'name': 'giselle avendaño',
    'id': 587704105,
    'id_str': '587704105',
    'indices': [39, 54]},
   {'screen_name': 'LorenaParisella',
    'name': 'Lorena Parisella',
    'id': 450023625,
    'id_str': '450023625',
    'indices': [55, 71

# User tags with tweepy

In [14]:
import tweepy

In [15]:
consumer_key = "dA6LwKzex4t2GqtYXKtZNQpxV" #1
consumer_secret = "tHsh8Vi1CHuOlgToTHggA2m2FPiZf8Lq9rnpmAK1uZa44QeFny" #2
access_token = "126698065-HU8BhbMUWkSAhfzUzS5wXeJW2Aiku2mfKyj1X3vV" #3
access_token_secret = "ZXrkqHvW8N4jt6SURKYyY9WhzASdQdUVZaxGZmWOX7kfH" #4
# User data
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# With this object we will call the Twitter API
api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [16]:
def tweet_loc_find (search_id):
    test_status = api.get_status(search_id, tweet_mode='extended')._json 
    return(test_status)

In [193]:
for k in range(len(tweets_df['Id'])):
    test = tweet_loc_find(tweets_df['Id'][k])
    tweets_df['user_id'][k] = test['user']['id']
    tweets_df['user_location'][k] = test['user']['location']

<ipython-input-193-95095fed6d33>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['user_id'][k] = test['user']['id']
<ipython-input-193-95095fed6d33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['user_location'][k] = test['user']['location']
C:\Users\joseph\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [194]:
tweets_df

,Id,text,json,location
0,0,0,0,0
1,429478735541460992,RT @Veneco_Babes: @marycarmenfh @trinysex @Div...,{'created_at': 'Sat Feb 01 04:58:04 +0000 2014...,None
2,429476144170422272,RT @Veneco_Babes: @marycarmenfh @trinysex @Div...,{'created_at': 'Sat Feb 01 04:47:46 +0000 2014...,None
3,429475247448793088,@aluxtuitea @rogeliosamour @JJRENDON @JNatanVa...,{'created_at': 'Sat Feb 01 04:44:12 +0000 2014...,None
4,429443798234824704,@Mayomendozaf @callollanos @JanioCuadro el que...,{'created_at': 'Sat Feb 01 02:39:14 +0000 2014...,None
...,...,...,...,...
11042,439722847704211456,RT @Veneca_NL: 'Beschamende hoeveelheden voeds...,{'created_at': 'Sat Mar 01 11:24:31 +0000 2014...,None
11043,439658737545256960,RT @lcvelez: Primera vez que un agente de migr...,{'created_at': 'Sat Mar 01 07:09:46 +0000 2014...,None
11044,439658698873798656,RT @lcvelez: Primera vez que un agente de migr...,{'created_at': 'Sat Mar 01 07:09:36 +0000 2014...,"(Venezuela, (6.68231285, -65.52259913185809))"
11045,439627929832923136,Feliz Cumple Manito!! Dios te de Vida y Salud!...,{'created_at': 'Sat Mar 01 05:07:21 +0000 2014...,None


# Geocoding

In [195]:
tweets_df['country']=0

In [196]:
geocoder = RateLimiter(Nominatim(user_agent='tweets_project').geocode, min_delay_seconds=1)

In [198]:
for k in range(len(tweets_df['Id'])):
    location = tweets_df['user_location'][k]
    tweets_df['country'][k] = geocoder(location)

<ipython-input-198-54972df37125>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['country'][k] = geocoder(location)
C:\Users\joseph\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\joseph\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the n

In [200]:
tweets_df.tail()

,index,Id,text,user_id,user_location,country
204,205,428228775143559168,Troféu praça de Venecos vai para Arthur.,304130084,,None
205,206,428227797593497600,@Bebe_Bastardo @hiletrado ve su tl jajaja puto...,340058954,"ATLAS, Jalisco","(Atlas Colomos, Zapopan, Jalisco, 45054, Méxic..."
206,207,428216729953525760,"A los comerciantes, porq los d migración son r...",126425918,Here & There #Greendale,None
207,208,428213812765151233,"""La Old School Regreso"" De @laraarmada Ft. Ven...",262510392,Venezuela,"(Venezuela, (6.68231285, -65.52259913185809))"
208,209,428211137143136257,@Lil_Veneca text message me dont imessage,379304181,from hell going to HEAVEN,None


In [160]:
test_status = tweet_loc_find(561746178300452865)#tweets_df['Id'][6])

In [161]:
if test_status['user']['id']: print(test_status['user']['id'])
if test_status['user']['followers_count']: print(test_status['user']['followers_count'])
if test_status['user']['friends_count']: print(test_status['user']['friends_count'])
if test_status['user']['listed_count']: print(test_status['user']['listed_count'])
if test_status['user']['location']: print(test_status['user']['location'])
if test_status['user']['geo_enabled']: print(test_status['user']['geo_enabled'])
if test_status['geo']: print(test_status['geo'])
if test_status['coordinates']: print(test_status['coordinates'])
if test_status['place']: print(test_status['place'])


#if test_status['retweeted_status']['id']: test_status['retweeted_status']['id']

459578065
223
275
tambogrande,piura
True
{'type': 'Point', 'coordinates': [-4.92883692, -80.34812358]}
{'type': 'Point', 'coordinates': [-80.34812358, -4.92883692]}
{'id': '0138ff3460a363e8', 'url': 'https://api.twitter.com/1.1/geo/id/0138ff3460a363e8.json', 'place_type': 'city', 'name': 'Tambo Grande', 'full_name': 'Tambo Grande, Peru', 'country_code': 'PE', 'country': 'Peru', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-80.6286340785714, -5.14701504272033], [-80.0755470016092, -5.14701504272033], [-80.0755470016092, -4.68941004442413], [-80.6286340785714, -4.68941004442413]]]}, 'attributes': {}}


In [159]:
test_status

{'created_at': 'Sat Feb 01 02:06:37 +0000 2014',
 'id': 429435589205585920,
 'id_str': '429435589205585920',
 'full_text': 'Recuerdos de mis bellos sobrino Venecos http://t.co/mza29GnMoJ',
 'truncated': False,
 'display_text_range': [0, 62],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'http://t.co/mza29GnMoJ',
    'expanded_url': 'http://fb.me/3lRontXRU',
    'display_url': 'fb.me/3lRontXRU',
    'indices': [40, 62]}]},
 'source': '<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 887897286,
  'id_str': '887897286',
  'name': 'LUZ CARRENO',
  'screen_name': 'lucecita4life',
  'location': 'ESTADOS UNIDOS',
  'description': 'HOLA AMIGOS: SOY DE COLOMBIA Y VIVO EN USA. SOY CNA Y DISTRIBUIDOR AUTORIZADO DE 4LIFE RESEARCH',
  'url': 'https://t.co